# Test et valuation of multiple supervised classification solution
The dataset is [basics statistics](https://leagueoflegends.fandom.com/wiki/List_of_champions/Base_statistics) of [League of Legend](https://www.leagueoflegends.com/fr-fr/) champions.

The 2 targets are :
  - recommanded [lanes](https://leagueoflegends.fandom.com/wiki/Lanes) for the champion in normal game mode (defines on which of the 5 lanes the champion is suited to play), encoded in 5 sub-targets.
  - champion's [class](https://leagueoflegends.fandom.com/wiki/Champion_classes) (define the general role of the champion)

## sommaire
- [install/import](#install--import)
- [Data preprocessing](#dataPreprocessing)
  - Feature Scaling
  - Feature selection
- Classification
  - KNN
    - KNN training
    - KNN valuation
  - Naive Bayes
    - Naive Bayes training
    - Naive Bayes valuation
  - Decision Trees
    - Decision Trees training
    - Decision Trees valuation
- Conclusion
- Classification of custom entity 

## install / import

In [6]:
#%pip install scikit-learn
#%pip install numpy
#%pip install pandas
import numpy as np
import pandas as pd

## Data preprocessing
Get the dataset and quick check of eventual failure.
- Features presentation:
  - champion name, useless for classification but more readable than entity numbers for graphical representation 
  - variables annotated with '+' indicate the scalling value added to the initial one for each champion level, from 1 to 18 over the course of the game.
  - HP : number of health points.
    - HP5 : number of health points recovered per 5 seconds
  - MP : Mana points, or energy point for some champs
    - MP5 : number of mana/energy points recovered per 5 seconds
  - AD : Attack Damage, physical pre-mitigation damage dealt when using basic attack (right click)
  - AS : Attack Speed, maximum attack the champ can deal per second 
    - the "+" scalling value is a percentage add to the implied "Bonus Attack Speed" value. It result that the **effective AS** = **basic AS***(1+**Bonus AS**) 
  - AR : ARmor, parameter of a non-linear function that calculates physical incurred damage (physical post-mitigated damage) as a function of physical pre-mitigation damage
  - MR : Magic Resistance, same as armor but for magic damages.
  - MS : Movement Speed, number of game-distance unit traveled per second
  - ranged : variable affecting access to and use of certain in-game objects.
  - range : radius around the champion in which basic attacks can be made.

  "Ranged" is the only non-numerical var but is directly linked to the "range" one, it's used to avoid confusion between certain champions: for example, Urgot is ranged but with a small 350 range for his category, while Lillia is not ranged, even though she has a range of 325. This variable is really important, as some in-game items are only available for one of the two categories.

Target presentation  
- Lanes :
  - "Top" :
  - "Jgl" :
  - "Mid" :
  - "Bot" :
  - "Sup" :
- Class : 
  - define the general role of the champion  
  - "marksman"   : use their long range damage and basic attacks to dps (deal constant Damage Per Second) enemies 
  - "assassin"   : able to penetrate enemy defenses to eliminate weak targets, thanks to their high mobility and capacity to avoid incoming damage
  - "burst"      : mage whose aim is to eliminate one or two targets with a combo of a few spells, then stand back
  - "diver"      : able to penetrate enemy defenses to eliminate weak targets, thanks to their high mobility 
  - "vanguard"   : lead the charge for their team and specialize at bringing the action, by high resistance, mobility and control capabilities
  - "specialist" : psychological zoners, control enemy pathing by using special positioning or zone spells to dissuade an opponent from approaching, at the risk of exposing themselves to a violent counter-attack
  - "juggernaut" : melee titans who excel at both dealing and taking significant amounts of damage, but get low ange and bad mobility
  - "skirmisher" : also named Duelist, built to win a 1v1 against any enemy and survive if others arrive, thanks to strong defensive or holding capabilities
  - "battlemage" : mage very efficient in short range damage areas
  - "enchanter"  : amplifying their allies' effectiveness, by healing or boost them
  - "catcher"    : control enemy movements, but allies-dependent because of low tankiness or low damages
  - "warden"     : defensive tanks. Wardens stand steadfast, seeking to hold the line by persistently locking down any on-comers who try to pass them.
  - "artillery"  : mage who excel in long range magic damage

In [18]:
df = pd.read_csv("./projetStatsChampsLol.csv")
df.head
df.Class.value_counts()

Class
marksman      25
assassin      17
burst         16
diver         16
vanguard      15
specialist    15
juggernaut    14
skirmisher    14
battlemage    11
enchanter      9
catcher        7
warden         6
artillery      6
Name: count, dtype: int64

**Check for missing values :**
no missing values.

In [13]:
df.isna().sum().sum()
#na

0

## Feature Scaling


In [9]:
qualitative_vars = ["Ranged"]
target_vars = ["isTop","isJgl","isMid","isBot","isSup","type"]
quantitative_vars = [[col for col in df.columns if col not in qualitative_vars + target_vars + ["Name"]]]
print(qualitative_vars)
print(quantitative_vars)
print(target_vars)

['Ranged']
[['HP', 'HP+', 'HP5', 'HP5+', 'MP', 'MP+', 'MP5', 'MP5+', 'AD', 'AD+', 'AS', 'AS+', 'AR', 'AR+', 'MR', 'MR+', 'MS', 'Range']]
['isTop', 'isJgl', 'isMid', 'isBot', 'isSup', 'type']


In [10]:
for col in df.column() :
    top = []
    top = df.sort_values()


AttributeError: 'DataFrame' object has no attribute 'column'